In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cancel deploy status

This notebook explains the workflow for canceling the deploy status.

Instructions are described in [README.md](./README.md).

In [1]:
import errno
import json
from pathlib import Path

import jsonschema
from console_access_library.client import Client

## Load Configurations

Load the configuration file and set the variables.

In [5]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration
jsonschema.validate(json_load, json_schema)

## Load an instance of "**Console Access Library**" client

Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [6]:
%store -r
client_obj: Client = client_obj

## Cancel deploy status

In [ ]:
# Get an instance of deploy API
deployment_obj = client_obj.get_deployment()

# Call an API to cancel deploy status
try:
    response = deployment_obj.cancel_deployment(
        device_id=json_load["device_id"], deploy_id=json_load["deploy_id"]
    )
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print("Deployment status changed to 'Cancel'.")